In [1]:
using DataFrames, CSV, Query
using GeoDataFrames, ArchGDAL
using Plots

In [ ]:
plotly()

In [ ]:
struct Point2D
	x::Float64
	y::Float64 
end

function draw_colored_points(x::Vector{Float64},y::Vector{Float64},labels::Array{Int64,2},plt::Plots.Plot)
	m = length(y)
	n = length(x)
	A = zeros(Int, m,n)
	for i=1:m 
		for j=1:n 
			A[i,j] = labels[i,j]
		end
	end
	return heatmap!(plt,x,y,A, c=:tab20c, colorbar=false)
end

function draw_points(v::Vector{Point2D},plt::Plots.Plot)
    x = [v[i].x for i=1:length(v)]
    y = [v[i].y for i=1:length(v)]
    scatter!(plt,x,y,legend=:false)
end

function voronoi(P::Vector{Point2D},metric=(A::Point2D,B::Point2D)->sqrt((B.x-A.x)^2+(B.y-A.y)^2);limx=[0,100],limy=[0,100],density=1000, mapf=nothing)
	px = [limx[1]:(limx[2]-limx[1])/density:limx[2];]
	py = [limy[1]:(limy[2]-limy[1])/density:limy[2];]
	npx = length(px)
	npy = length(py)
	n = length(P)
	points = Vector{Point2D}(undef, npx * npy)
	labels = zeros(Int64, npy, npx)
	
	for i=1:npx 
		for j=1:npy 
			curr_p = Point2D(px[i],py[j])

			(mapf != nothing) && (!mapf(curr_p)) && continue

			m = map(p -> metric(p, curr_p), P)
			minval, imin = findmin(m)

			points[(j - 1) * npx + i] = curr_p
			labels[j, i] = imin
			
		end
		
	end
	plt = plot()
	println("Drawing ... ")
	draw_colored_points(px, py, labels, plt)
	draw_points(P, plt)
	return plt
end

In [ ]:
cod_maringa = 4115200

In [ ]:
maringa = GeoDataFrames.read("data/municipios/PR_Municipios_2022.shp") |> @filter(_.CD_MUN == string(cod_maringa)) |> DataFrame

In [ ]:
ubs_brasil = CSV.read("data/ubs.csv.gz", DataFrame);

In [ ]:
ubs_maringa = ubs_brasil |> @filter(_.IBGE == cod_maringa / 10) |> @select(:CNES, :NOME, :geometry, :LONGITUDE, :LATITUDE) |> DataFrame

In [ ]:
ubs_maringa[!, "geometry"] = ArchGDAL.createpoint.(parse.(Float64, replace.(convert.(String, ubs_maringa.LONGITUDE), "," => ".")), parse.(Float64, replace.(convert.(String, ubs_maringa.LATITUDE), "," => ".")))

In [ ]:
using GeoFormatTypes

In [ ]:
c_origem = GeoFormatTypes.EPSG(4674);
c_destino = GeoFormatTypes.EPSG(5880);

In [ ]:
reproject(ubs_maringa[:, "geometry"], c_origem, c_destino, order=:trad);
reproject(maringa[:, "geometry"], c_origem, c_destino, order=:trad);

In [ ]:
plot(GeoDataFrames.boundary.(maringa.geometry), color=:black, aspectratio=true, grid=false)
plot!(ubs_maringa.geometry, marker=:circle, color=:green, markersize=3)

In [ ]:
P = [Point2D(ArchGDAL.getx(p, 0), ArchGDAL.gety(p, 0)) for p in ubs_maringa.geometry]

In [ ]:
p = ArchGDAL.envelope(maringa.geometry[1])

In [ ]:
voronoi(P; limx=[5.1940058483459605e6, 5.221342165950402e6], limy=[7.392234804831119e6, 7.425889009771498e6], density=500)
plot!(GeoDataFrames.boundary.(maringa.geometry), color=:black, aspectratio=true, grid=false)
xaxis!(false)
yaxis!(false)

In [ ]:
esta_no_mapa = let 

    pg = ArchGDAL.preparegeom(maringa.geometry[1])

    (p::Point2D) -> ArchGDAL.contains(pg, ArchGDAL.createpoint((p.x, p.y)))

end

In [ ]:
voronoi(P; limx=[5.1940058483459605e6, 5.221342165950402e6], limy=[7.392234804831119e6, 7.425889009771498e6], density=500, mapf=esta_no_mapa)
plot!(GeoDataFrames.boundary.(maringa.geometry), color=:black, aspectratio=true, grid=false)
xaxis!(false)
yaxis!(false)

In [ ]:
reproject(ubs_maringa[:, "geometry"], c_destino, c_origem, order=:trad);

# O problema

Primeiramente, adaptamos a nossa função de Voronoi para fazer duas tarefas: estimar a área de cada região poligonal e estimar a quantidade de maringaenses que mora em cada região poligonal. Para o primeiro, assumindo uma distribuição uniforme dos pontos, calculamos a relação entre os pontos que estão dentro de uma região `i` e a quantidade total de pontos. Para a segunda, simplesmente verificamos as distâncias entre os dados do IBGE e cada UBS.

In [ ]:
function voronoi_densidade(P::Vector{Point2D}, pibge::Vector{Point2D}, metric=(A::Point2D,B::Point2D)->sqrt((B.x-A.x)^2+(B.y-A.y)^2);limx=[0,100],limy=[0,100],density=1000, mapf=nothing)
	px = [limx[1]:(limx[2]-limx[1])/density:limx[2];]
	py = [limy[1]:(limy[2]-limy[1])/density:limy[2];]
	npx = length(px)
	npy = length(py)
	n = length(P)

	area_total = (limx[2]-limx[1])*(limx[2]-limx[1])
	pts_total = npx*npy 

    count = zeros(Int, n)

	for i=1:npx 
		for j=1:npy 
			curr_p = Point2D(px[i],py[j])

			(mapf != nothing) && (!mapf(curr_p)) && continue

			m = map(p -> metric(p, curr_p), P)
			minval, imin = findmin(m)

            count[imin] += 1
			
		end
		
	end

    # Estimativa da area usando pontos igualmente espacados
    area = [(count[i] / pts_total) * area_total for i in 1:n]

    # Quantidade de pessoas do IBGE que estao no poligono i
    pop = zeros(Int, n)

    for p in pibge

        m = map(x -> metric(x, p), P)
        minval, imin = findmin(m)

        pop[imin] += 1

    end

    # Densidade em habs /metro quadrado
    return pop ./ area
end